In [1]:
getwd()
library(glmnet)
library(caret)
library(dplyr)
library(plyr)
#install.packages('R.utils')
library(R.utils)

[1] "/home/sf673542/DataWrangling/AnalyzeData/AllBiopsies"

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16

Loading required package: lattice
Loading required package: ggplot2

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

Loading required package: R.oo
Loading required package: R.methodsS3
R.methodsS3 v1.7.1 (2016-02-15) successfully loaded. See ?R.methodsS3

In [2]:
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)

#### Creating main directory variables: 

In [3]:
mainDir = "/home/sf673542/DataWrangling/AnalyzeData/AllBiopsies/"
dataDir = "/home/sf673542/DataWrangling/AnalyzeData/AllBiopsies/"

#### Directories to save results: 

In [4]:
GLMM.missclass.dir <- paste("Misclassification_Results/GLMM_Output_Unbalanced/")
dir.create(file.path(mainDir,GLMM.missclass.dir), showWarnings = FALSE)

In [5]:
GLMM.logLoss.dir <- paste("Misclassification_Results/GLMM_Output_Unbalanced/")
dir.create(file.path(mainDir,GLMM.logLoss.dir), showWarnings = FALSE)

In [6]:
GLMM.AUC.dir <- paste("Misclassification_Results/GLMM_AUC_Unbalanced/")
dir.create(file.path(mainDir,GLMM.AUC.dir), showWarnings = FALSE)

In [7]:
GLMM.pred.dir <- paste("Predicted_Output/GLMM_predicted_Unbalanced/")
dir.create(file.path(mainDir,GLMM.pred.dir), showWarnings = FALSE)

In [8]:
GLMM.prob.dir <- paste("Probability_Output/GLMM_probability_Unbalanced/")
dir.create(file.path(mainDir,GLMM.prob.dir), showWarnings = FALSE)


In [9]:
GLMM.accuracy.dir <- paste("Accuracy_Results/GLMM_accuracy_Unbalanced/")
dir.create(file.path(mainDir,GLMM.accuracy.dir), showWarnings = FALSE)

In [10]:
GLMM.cm.dir <- paste("Confusion_Matrix/GLMM_cm_Unbalanced/")
dir.create(file.path(mainDir,GLMM.cm.dir), showWarnings = FALSE)

#### Give the simulation parameters:

In [11]:
model <- cmdArg(model = "GLMM_Unbalanced")
n.simulations <- cmdArg(n.simulations = 1000L)

In [12]:
n.simulations

[1] 1000

In [13]:
cat("Started GLMM Analysis ...\n")

Started GLMM Analysis ...


In [14]:
idx = 1

In [15]:
training.data = as.data.frame(read.csv(file = paste(dataDir,"Training/","train_data_",idx,".csv",sep = "")))

In [16]:
test.data = as.data.frame(read.csv(file = paste(dataDir,"Testing/","test_data_",idx,".csv",sep = "")))

In [17]:
varNames = names(training.data)
data.frame(index = 1:length(varNames), varNames = varNames)

index,varNames
1,b_number
2,t_number
3,sf_number
4,current_scan_date
5,current_surgery_date
6,current_hist_grade
7,current_hist_type
8,roi.label
9,tumor_cell_evaluation
10,necrosis


In [18]:
varNames_toPredict = varNames[c(16:21, 32, 34, 35, 36:41, 43, 65, 67, 68)]
varNames_toPredict
varNames.add.sign <- paste(varNames_toPredict, collapse = "+")
varNames.add.sign

[1] "nfse"        "nfl"         "nt1c"        "nt1v"        "nadc.1"     
 [6] "nfa.1"       "cbvn_nlin"   "phn_npar"    "recovn_npar" "cni"        
[11] "ccri"        "crni"        "ncho"        "ncre"        "nnaa"       
[16] "laclip"      "in_CEL"      "in_NEL"      "in_NEC"

[1] "nfse+nfl+nt1c+nt1v+nadc.1+nfa.1+cbvn_nlin+phn_npar+recovn_npar+cni+ccri+crni+ncho+ncre+nnaa+laclip+in_CEL+in_NEL+in_NEC"

### Normalize predictor variables: 

In [27]:
str(training.data[,varNames_toPredict])
varNames_forNormalization = varNames_toPredict[-c(17:19)]
varNames_forNormalization

'data.frame':	294 obs. of  19 variables:
 $ nfse       : num  2.1 2.27 2.35 2.2 0.85 1.08 2.16 1.85 2.56 1.72 ...
 $ nfl        : num  1.84 1.44 1.97 1.56 0.66 1.23 1.74 1.12 2.16 1.11 ...
 $ nt1c       : num  1.2 1.88 0.76 2.04 1.99 0.95 1.94 0.92 0.81 2.31 ...
 $ nt1v       : num  0.84 0.78 0.78 0.74 NA NA 0.9 NA 0.85 1.14 ...
 $ nadc.1     : num  1.31 1.92 1.34 2.04 1.14 1.3 1.75 1.43 1.76 1.18 ...
 $ nfa.1      : num  0.75 0.68 0.68 0.89 NA NA 0.78 NA 0.54 0.96 ...
 $ cbvn_nlin  : num  0.66 2.24 0.58 1.71 0.78 1.09 1.36 1.51 0.4 3.73 ...
 $ phn_npar   : num  0.79 2.25 0.6 1.88 0.77 0.88 1.36 1.58 0.42 3.39 ...
 $ recovn_npar: num  100 98.5 100 99.2 100 ...
 $ cni        : num  4.98 1.56 NA 2.46 NA NA NA NA 4.28 NA ...
 $ ccri       : num  3.28 0.3 NA 1.3 NA NA NA NA 2.28 NA ...
 $ crni       : num  0.24 0.75 NA 0.55 NA NA NA NA 0.64 NA ...
 $ ncho       : num  1.65 0.86 NA 1.19 NA NA NA NA 1.32 NA ...
 $ ncre       : num  0.53 0.84 NA 0.87 NA NA NA NA 0.56 NA ...
 $ nnaa       : nu

[1] "nfse"        "nfl"         "nt1c"        "nt1v"        "nadc.1"     
 [6] "nfa.1"       "cbvn_nlin"   "phn_npar"    "recovn_npar" "cni"        
[11] "ccri"        "crni"        "ncho"        "ncre"        "nnaa"       
[16] "laclip"

In [28]:
normalized.training.data <- cbind(as.data.frame(scale(training.data[,varNames_forNormalization])))
summary(normalized.training.data)

      nfse              nfl                 nt1c              nt1v         
 Min.   :-2.3365   Min.   :-3.318044   Min.   :-2.4054   Min.   :-2.22044  
 1st Qu.:-0.5428   1st Qu.:-0.620150   1st Qu.:-0.7055   1st Qu.:-0.72122  
 Median :-0.1066   Median :-0.009677   Median :-0.2033   Median :-0.03975  
 Mean   : 0.0000   Mean   : 0.000000   Mean   : 0.0000   Mean   : 0.00000  
 3rd Qu.: 0.4371   3rd Qu.: 0.669720   3rd Qu.: 0.5887   3rd Qu.: 0.50542  
 Max.   : 9.2861   Max.   : 4.322707   Max.   : 3.7953   Max.   : 5.50282  
                   NA's   :3                             NA's   :54        
     nadc.1             nfa.1           cbvn_nlin          phn_npar      
 Min.   :-2.91856   Min.   :-2.5238   Min.   :-0.8308   Min.   :-1.4161  
 1st Qu.:-0.63760   1st Qu.:-0.6684   1st Qu.:-0.4660   1st Qu.:-0.6574  
 Median :-0.04955   Median :-0.1652   Median :-0.2112   Median :-0.2605  
 Mean   : 0.00000   Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.0000  
 3rd Qu.: 0.48759   3r

In [ ]:
for (idx in 1:n.simulations){
  tic("Total Execution time")
  set.seed(idx)
  
  #Load training and validation data
  training.data <- as.data.frame(read.csv(file = paste(dataDir,"Training/","train_data_",idx,".csv",sep = "")))
  test.data <- as.data.frame(read.csv(file = paste(dataDir,"Testing/","test_data_",idx,".csv",sep = "")))
  
  #Create formula for Logreg
  varNames <- names(training.data)
  varNames <- varNames[!varNames %in% c("Outcome")]
  varNames.add.sign <- paste(varNames, collapse = "+")
  lasso.formula <- as.formula(paste("Outcome",varNames.add.sign, sep = "~"))
  
  
  #Normalize predictor variables
  normalised.training.data <- cbind(as.data.frame(scale(training.data[,-length(training.data)])),training.data[,length(training.data)])
  colnames(normalised.training.data)[length(normalised.training.data)] <- "Outcome"
  
  normalised.test.data <- cbind(as.data.frame(scale(test.data[,-length(test.data)])),test.data[,length(test.data)])
  colnames(normalised.test.data)[length(normalised.test.data)] <- "Outcome"
  
  
  # Create model matrix for train & test set
  norm_train_mtx <- model.matrix(lasso.formula, normalised.training.data)[, -1]
  norm_train_outcome <- normalised.training.data$Outcome
  
  norm_test_mtx <- model.matrix(lasso.formula, normalised.test.data)[, -1]
  norm_test_outcome <- normalised.test.data$Outcome
  
  ################################################### LASSO ######################################################################
  # Building Lasso Model
  cv.lasso <- cv.glmnet( norm_train_mtx,
                        norm_train_outcome,
                        alpha=1,family = "binomial",standardize = FALSE)
  
  bestlam.lasso <- cv.lasso$lambda.min
  
  reg.lasso  <- glmnet(norm_train_mtx,
                      norm_train_outcome,
                      alpha=1,
                      lambda= bestlam.lasso,
                      family = "binomial",
                      standardize = FALSE)
  ############################################## TRAINING ENDS #########################################################################
  pred.lasso.train <- predict(reg.lasso,s= bestlam.lasso ,newx= norm_train_mtx, type = "class")
  
  pred.lasso <- predict(reg.lasso,s= bestlam.lasso ,newx= norm_test_mtx, type = "class")
  pred.lasso.prob <- predict(reg.lasso,s= bestlam.lasso ,newx= norm_test_mtx, type ="response")
  
  coef.lasso <- predict(reg.lasso , type="coefficients", s=bestlam.lasso)
  misclass.err  <- mean(norm_test_outcome != pred.lasso)
  
  Confusion_Matrix_Train <- confusionMatrix(data = pred.lasso.train,reference =  norm_train_outcome,positive = "M")
  Confusion_Matrix_Test <- confusionMatrix(data = pred.lasso,reference =  norm_test_outcome,positive = "M")
  
  
  # LogLoss error
  source("/home/pchunduru/Simulation_data/Classification/Scripts/functions/logLoss.R")
  df_pred_probs <- data.frame(B= 1-pred.lasso.prob, 
                              M = pred.lasso.prob)
  prediction_test_logloss <- apply(df_pred_probs, c(1,2), function(x) min(max(x, 1E-15), 1-1E-15))
  logloss_err <- logLoss(prediction_test_logloss, norm_test_outcome)
  
  #Data frame of Truth and Predicted values
  predicted.test.set <- cbind(test.data[,-length(test.data)],pred.lasso)
  colnames(predicted.test.set)[length(test.data)] <- c("Pred.Outcome")
  
  probability.test.set <-cbind(test.data[,-length(test.data)], pred.lasso.prob)
  colnames(probability.test.set)[length(test.data)] <- c("Prob.Outcome.M")
  
  
  #Get AUC value
  require(ROCR)
  require(pROC)
  perf_ROC <- roc(norm_test_outcome ,pred.lasso.prob[,1])
  auc_output <- pROC::auc(perf_ROC)

  ## Saving the results
  # missclassification error
  write(missclass.err, paste(mainDir,Logreg.missclass.dir,model,"_","ErrorToTruth_",idx,".txt",sep=""),ncol=1)
  # LogLoss error
  write(logloss_err, paste(mainDir,Logreg.logLoss.dir,model,"_","logLoss_",idx,".txt",sep=""),ncol=1)
  #AUC values
  write(auc_output[1], paste(mainDir,Logreg.AUC.dir,model,"_","AUC_",idx,".txt",sep=""),ncol=1)
  #predicted output data
  write.csv(x = predicted.test.set,file = paste(mainDir,Logreg.pred.dir,model,"_","Predicted_Test_Set_",idx,".csv",sep=""),row.names = F,col.names = T,sep = "\t")
  #predicited probabilities result
  write.csv(x = probability.test.set ,file = paste(mainDir,Logreg.prob.dir,model,"_","Probability_Test_Set_",idx,".csv",sep=""),row.names = F,col.names = T,sep = "\t")
  # confusion matrix
  write.csv(x = Confusion_Matrix_Test$byClass , file =paste(mainDir,Logreg.cm.dir,model,"_","Confusion_Matrix_",idx,".csv",sep=""),row.names = T,col.names = T,sep = "\t")
  # Accuracy
  write.csv(x = Confusion_Matrix_Test$overall , file =paste(mainDir,Logreg.accuracy.dir,model,"_","Accuracy_",idx,".csv",sep=""),row.names = T,col.names = T,sep = "\t")
  #
}